In [124]:
# here, we combine all .csv files into the single file total_state_data_monthly.csv

import pandas as pd
import numpy as np

# df_pop = pd.read_csv('../SharedData/dataset-generation-final/population-1990-2020-final.csv')
# df_emissions = pd.read_csv('../SharedData/dataset-generation-final/monthly-emissions-1990-2024-final.csv')
# df_weather = pd.read_csv('../SharedData/dataset-generation-final/monthly-weather-1990-2019-final.csv')
# df_gdp = pd.read_csv('../SharedData/dataset-generation-final/gdp-1997-2023-final.csv')
# df_energy = pd.read_csv('../SharedData/dataset-generation-final/energy-use-prod-final.csv')
# df_pp = pd.read_csv('../SharedData/dataset-generation-final/power-plant-count-final.csv')
# df_area = pd.read_csv('../SharedData/dataset-generation/state-areas.csv')
# df_esource = pd.read_csv('../SharedData/dataset-generation-final/energy-by-source-final.csv')
# df_votes = pd.read_csv('../SharedData/dataset-generation-final/republican-votes-by-state.csv')

df1 = pd.read_csv('../SharedData/dataset-generation-final/population-1990-2020-final.csv')
df2 = pd.read_csv('../SharedData/dataset-generation-final/monthly-emissions-1990-2024-final.csv')
df3 = pd.read_csv('../SharedData/dataset-generation-final/monthly-weather-1990-2019-final.csv')
df4 = pd.read_csv('../SharedData/dataset-generation-final/gdp-1997-2023-final.csv')
df5 = pd.read_csv('../SharedData/dataset-generation-final/energy-use-prod-final.csv')
df6 = pd.read_csv('../SharedData/dataset-generation-final/power-plant-count-final.csv')
df7 = pd.read_csv('../SharedData/dataset-generation-final/energy-by-source-final.csv')
df8 = pd.read_csv('../SharedData/dataset-generation-final/republican-votes-by-state.csv')
df9 = pd.read_csv('../SharedData/dataset-generation-final/state-areas-final.csv')


In [125]:
# merge all dfs 1-8 on 'state' and 'date'
total_df = df1.merge(df2, on=['state','date'], how='right').merge(df3, on=['state','date'], how='left').merge(df4, on=['state','date'], how='left').merge(df5, on=['state','date'], how='left').merge(df6, on=['state','date'], how='left').merge(df7,on=['state','date'], how='left').merge(df8,on=['state','date'], how='left') 

# remove DC, PR (not states) and HI, AK (states with missing CO2 data)
total_df = total_df[~total_df.state.isin(['DC','PR','HI','AK'])]
assert(len(total_df.state.unique() == 48))

# ensure the 'date' column is in correct datetime formatting
total_df['date']=pd.to_datetime(total_df['date'])

In [126]:
mo_cols = ['state', 'date',
           'monthly_emissions', 'monthly_population', 
           'prcp', 'snow', 'tavg', 
           'monthly_gdp_rel_2017', 
           'monthly_energy_prod', 'monthly_energy_use', 'monthly_energy_flow', 
           'monthly_num_plants',
           'total_facility_age', 
           'monthly_renew_pct', 'monthly_fossil_pct', 
           'republican_vote_pct']

yr_cols = ['state', 'date',
           'monthly_emissions', 'yearly_population',
           'prcp', 'snow', 'tavg', 
           'yearly_gdp_rel_2017', 
           'yearly_energy_prod', 'yearly_energy_use', 'yearly_energy_flow', 
           'yearly_num_plants', 
           'total_facility_age', 
           'yearly_renew_pct', 'yearly_fossil_pct',
           'republican_vote_pct']

monthly_total_df = total_df[mo_cols]
yearly_total_df = total_df[yr_cols]

Now we create some additional columns:

In [127]:
df9_dict = df9.set_index('state')['area'].to_dict()

# add columns for monthly/yearly population density
monthly_total_df['monthly_pop_density'] = monthly_total_df['state'].map(df9_dict) / monthly_total_df['monthly_population']
yearly_total_df['yearly_pop_density'] = yearly_total_df['state'].map(df9_dict) / yearly_total_df['yearly_population']

C:\Users\lmwhe\AppData\Local\Temp\ipykernel_14724\1725428087.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_total_df['monthly_pop_density'] = monthly_total_df['state'].map(df9_dict) / monthly_total_df['monthly_population']
C:\Users\lmwhe\AppData\Local\Temp\ipykernel_14724\1725428087.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yearly_total_df['yearly_pop_density'] = yearly_total_df['state'].map(df9_dict) / yearly_total_df['yearly_population']


In [128]:
# add all per capita features:

monthly_total_df['co2_per_capita'] = monthly_total_df['monthly_emissions'] / monthly_total_df['monthly_population']
monthly_total_df['gdp_per_capita'] = monthly_total_df['monthly_gdp_rel_2017'] / monthly_total_df['monthly_population']
monthly_total_df['eprod_per_capita'] = monthly_total_df['monthly_energy_prod'] / monthly_total_df['monthly_population']
monthly_total_df['eflow_per_capita'] = monthly_total_df['monthly_energy_flow'] / monthly_total_df['monthly_population']
monthly_total_df['euse_per_capita'] = monthly_total_df['monthly_energy_use'] / monthly_total_df['monthly_population']
monthly_total_df['avg_facility_age'] = monthly_total_df['total_facility_age'] / monthly_total_df['monthly_num_plants']

# yearly_total_df['co2_per_capita'] = yearly_total_df['yearly_emissions'] / yearly_total_df['yearly_population']
yearly_total_df['co2_per_capita'] = yearly_total_df['monthly_emissions'] / yearly_total_df['yearly_population']
yearly_total_df['gdp_per_capita'] = yearly_total_df['yearly_gdp_rel_2017'] / yearly_total_df['yearly_population']
yearly_total_df['eprod_per_capita'] = yearly_total_df['yearly_energy_prod'] / yearly_total_df['yearly_population']
yearly_total_df['eflow_per_capita'] = yearly_total_df['yearly_energy_flow'] / yearly_total_df['yearly_population']
yearly_total_df['euse_per_capita'] = yearly_total_df['yearly_energy_use'] / yearly_total_df['yearly_population']
yearly_total_df['avg_facility_age'] = yearly_total_df['total_facility_age'] / yearly_total_df['yearly_num_plants']


C:\Users\lmwhe\AppData\Local\Temp\ipykernel_14724\2427158815.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_total_df['co2_per_capita'] = monthly_total_df['monthly_emissions'] / monthly_total_df['monthly_population']
C:\Users\lmwhe\AppData\Local\Temp\ipykernel_14724\2427158815.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_total_df['gdp_per_capita'] = monthly_total_df['monthly_gdp_rel_2017'] / monthly_total_df['monthly_population']
C:\Users\lmwhe\AppData\Local\Temp\ipykernel_14724\2

In [131]:
# Rearrange cols for readability:
monthly_total_df = monthly_total_df[['state', 'date', 
                                     'monthly_emissions', 'co2_per_capita',
                                     'prcp', 'snow', 'tavg', 
                                     'monthly_gdp_rel_2017', 'gdp_per_capita',
                                     'monthly_energy_prod', 'eprod_per_capita', 
                                     'monthly_energy_use', 'eflow_per_capita', 
                                     'monthly_energy_flow', 'euse_per_capita',
                                     'monthly_num_plants', 
                                     'monthly_renew_pct', 'monthly_fossil_pct',
                                     'republican_vote_pct', 
                                     'monthly_pop_density', 
                                     'avg_facility_age']]

monthly_capita_df  = monthly_total_df[['state', 'date', 
                                     'co2_per_capita',
                                     'prcp', 'snow', 'tavg', 
                                     'gdp_per_capita',
                                     'eprod_per_capita', 
                                     'eflow_per_capita', 
                                     'euse_per_capita',
                                     'monthly_num_plants', 
                                     'monthly_renew_pct', 'monthly_fossil_pct',
                                     'republican_vote_pct', 
                                     'monthly_pop_density', 
                                     'avg_facility_age']]


yearly_total_df = yearly_total_df[['state', 'date', 
                                     'monthly_emissions', 'co2_per_capita',
                                     'prcp', 'snow', 'tavg', 
                                     'yearly_gdp_rel_2017', 'gdp_per_capita',
                                     'yearly_energy_prod', 'eprod_per_capita', 
                                     'yearly_energy_use', 'eflow_per_capita', 
                                     'yearly_energy_flow', 'euse_per_capita',
                                     'yearly_num_plants', 
                                     'yearly_renew_pct', 'yearly_fossil_pct',
                                     'republican_vote_pct', 
                                     'yearly_pop_density', 
                                     'avg_facility_age']]

yearly_capita_df = yearly_total_df[['state', 'date', 
                                     'co2_per_capita',
                                     'prcp', 'snow', 'tavg', 
                                     'gdp_per_capita',
                                     'eprod_per_capita', 
                                     'eflow_per_capita', 
                                     'euse_per_capita',
                                     'yearly_num_plants', 
                                     'yearly_renew_pct', 'yearly_fossil_pct',
                                     'republican_vote_pct', 
                                     'yearly_pop_density', 
                                     'avg_facility_age']]

TODO: Add SMA calculations?

In [ ]:
# sma_feature_list = ['monthly_pop_density', 'yearly_pop_density',
#        'monthly_emissions', 'prcp', 'snow', 'tavg',
#        'gdp_rel_2017', 'gdp_rel_2017_interp', 'yearly_gdp_rel_2017',
#        'monthly_energy_prod', 'yearly_energy_prod', 'monthly_energy_use',
#        'yearly_energy_use', 'monthly_energy_flow', 'yearly_energy_flow',
#        'monthly_num_plants', 'yearly_num_plants', 'total_facility_age',
#        'monthly_energy_total', 'yearly_energy_total', 'monthly_energy_renew',
#        'yearly_energy_renew', 'monthly_renew_pct', 'yearly_renew_pct',
#        'monthly_energy_fossil', 'yearly_energy_fossil', 'monthly_fossil_pct',
#        'yearly_fossil_pct', 'co2_per_capita',
#        'gdp_per_capita', 'eprod_per_capita', 'eflow_per_capita',
#        'euse_per_capita', 'avg_facility_age']

# # create column for simple moving avg of co2_per_capita; ma_window is the moving average window size (n=1 corresponds to usual monthly results)

# df_mo['co2_per_capita_sma']= np.nan
# ma_window = 12

# for feature in sma_feature_list:
#     for state in abbreviations:
#         df_mo.loc[df_mo.state==state,feature+str('_sma')] = df_mo.loc[df_mo.state==state,feature].rolling(window=ma_window).mean()


In [132]:
monthly_total_df.to_csv('../SharedData/FinalData/monthly_total_data.csv', index=False)
monthly_capita_df.to_csv('../SharedData/FinalData/monthly_capita_data.csv', index=False)
yearly_total_df.to_csv('../SharedData/FinalData/yearly_total_data.csv', index=False)
yearly_capita_df.to_csv('../SharedData/FinalData/yearly_capita_data.csv', index=False)